# Lung Cancer Subtyping
a mathematical model that can discriminate between two lung cancer subtypes to build the model 
we use an unsupervised k-means clustering algorithm (Euclidean distance) of 58 NSCLC tumors using k=2

data from: 

https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE10245

[GSE10245_family.soft.gz](https://ftp.ncbi.nlm.nih.gov/geo/series/GSE10nnn/GSE10245/soft/)


In [ ]:
# load the data using the GEOparse module
